## 謹慎使用Python繼承(ABC)
---

+ 增加不必要的複雜性
  - 使用 ABC 需要額外的 abc 模組（from abc import ABC, abstractmethod），並且要顯式標註 @abstractmethod，對於一些 Pythonic（符合 Python 風格）的方法來說，這可能顯得繁瑣。

+ Python 是動態語言
  - Python 本身是一種動態類型語言，不強制類型檢查。因此，使用 ABC 來強制子類實作某些方法，與 Python 的靈活特性不太契合。

+ 影響靈活性
  - 如果某個類繼承 ABC，但只需要部分功能，仍然必須實作所有抽象方法，否則無法實例化。這在某些場景下可能不太合適。

+ Duck Typing 更加 Pythonic
  - Python 提倡 Duck Typing（鴨子類型），即「如果它像鴨子、會呱呱叫，那它就是鴨子」。強制使用 ABC 來定義結構，反而可能違反這種靈活性。

## 替代方法
1. 使用 Duck Typing
> Duck Typing 的概念是，只要物件具有某些方法或屬性，Python 會假設它是可用的，而不需要強制繼承特定的類別。

In [2]:
class Dog:
    def speak(self):
        return "Woof!"

class Cat:
    def speak(self):
        return "Meow!"

def animal_sound(animal):  # 缺點是animal沒有限制類型 (這是動態語言的特性，天生多態，又不是Java的多態)
    return animal.speak()  # 只要有 speak 方法，這個函數就能正常運作

print(animal_sound(Dog()))  # Woof!
print(animal_sound(Cat()))  # Meow!

Woof!
Meow!


2. 使用Protocol
> 如果你仍然希望提供某種「結構化」的方式來定義介面，但又不想強制繼承，Python 3.8+ 提供了 Protocol（來自 typing 模組），這是一種更靈活的方式(像Java的interface)。


In [3]:
from typing import Protocol

class Speakable(Protocol):
    def speak(self) -> str:
        pass

class Dog:
    def speak(self):
        return "Woof!"

class Cat:
    def speak(self):
        return "Meow!"

def animal_sound(animal: Speakable): #與鴨子類型的差異是這裡限制了傳入類型
    return animal.speak()

print(animal_sound(Dog()))  # Woof!
print(animal_sound(Cat()))  # Meow!


Woof!
Meow!


3. 使用Mixin
> 如果你的目標是提供某些行為給多個類，但不希望強制繼承，可以使用 Mixin 模式，使用上Mixin不要有初始化方法__init__，應用在多個子類都有相同的工具包這樣的概念。

In [1]:
class LoggerMixin:
    def log(self, message):
        print(f"[LOG]: {message}")

class Dog(LoggerMixin):
    def speak(self):
        self.log("Dog is speaking")
        return "Woof!"

dog = Dog()
print(dog.speak())  # [LOG]: Dog is speaking  Woof!


[LOG]: Dog is speaking
Woof!


## 總結

<style>
td,th {
  font-size: 18px;
}
</style>

|       方法        |               優勢            |                     何時使用                      |
|:---------------:|:------------------------------:|:---------------------------------------------:|
|    ABC（抽象基類）|  強制子類實作某些方法，確保結構一致 |          當你希望所有子類都有相同的 API，並且要在編譯時檢查          |
|   Duck Typing   |          Pythonic，靈活         |          當你只關心對象是否具有特定方法，而不管它的類別繼承關係          |
|    Protocol     |          兼顧靜態類型檢查與靈活性  |           需要靜態類型檢查（如 mypy），但不希望強制繼承           |
|      Mixin      |       提供可選行為，而不是強制結構  |             需要為多個類別提供額外功能但不強制 API             |
